#### **NEM Review contract co-design workshop**

# Contract financial performance modelling

## Prepare data

### Access generators from latest Generation Information

In [1]:
# Python libraries
import matplotlib.pyplot as plt, matplotlib as mpl, numpy as np
import os, pandas as pd, seaborn as sns, statsmodels.api as sm
from sqlalchemy import create_engine, text
from Scripts.functions import python_setup, get_mms_data

# Set up Python
working_dir, charts_dir, data_dir = python_setup()

# Get generator details
gen_info = pd.read_excel(
    os.path.join(data_dir, "NEM Generation Information July 2025.xlsx"),
    sheet_name=4,
    header=1,
    usecols=[0, 1, 2, 5, 6],
    names=["Region", "Status", "Name", "Technology", "DUID"])

# Filter and reformat data
gen_info = gen_info[    
    # Existing stations only
    (gen_info["Status"] == "Existing Plant") & 
    # SA only
    (gen_info["Region"] == "SA1") & 
    # Wind and solar only
    (gen_info["Technology"].isin(["Wind - Wind", "Solar - Solar"]))
].iloc[:, [2, 3, 4]]
# Simplify technology name
gen_info["Technology"] = gen_info["Technology"].map(lambda t: t.split(" - ")[1])

# Remove duplicate DUIDs
gen_info.drop_duplicates(subset="DUID", keep="first", inplace=True)

# Reorder by DUID
gen_info.set_index("DUID", inplace=True)

# Display DUIDs used in analysis
for tech in gen_info["Technology"].value_counts().index:
    print(f"{tech}: {gen_info["Technology"].value_counts().loc[tech]} generators")

ModuleNotFoundError: No module named 'Scripts.functions'

### Collect historic DUID data for South Australia

If no data yet exists, execute SQL query to collect output, availability and maximum capacity data for each DUID.

In [ ]:
# SA DUID data path
sa_duid_data_path = os.path.join(data_dir, "sa_duid_data.csv")

# Check if file exists
if not os.path.exists(sa_duid_data_path):
    print("sa_duid_data.csv not found. Querying database ...")

    # Define DUIDs of interest
    duid_list = gen_info.index

    # Get generation and price data
    sa_duid_data = get_mms_data(
        script_name="collect_sa_duid_data",
        arguments={
            "start_date": "2020-01-01",
            "end_date": "2024-12-31",
            "duid_list": ",".join([f"'{d}'" for d in duid_list])})

    # Save data to file
    sa_duid_data.to_csv(sa_duid_data_path, index=False)    
    print(f"Data saved successfully.")

else:
    print(f"Loading existing sa_duid_data.csv file ...")
    # Load existing data
    sa_duid_data = pd.read_csv(
        sa_duid_data_path,
        parse_dates=[0])

# Add name and technology
print("Joining with gen info data ...")
sa_duid_data["Technology"] = sa_duid_data["DUID"].map(lambda d: gen_info.at[d, "Technology"])
sa_duid_data["Name"] = sa_duid_data["DUID"].map(lambda d: gen_info.at[d, "Name"])

# Display market data
print("Data loaded successfully.")
sa_duid_data

### Collect whole-of-market data for South Australia

If no data yet exists, execute SQL query to collect SA prices and total demand data.

In [ ]:
# SA demand and prices path
sa_demand_and_prices_path = os.path.join(data_dir, "sa_demand_and_prices.csv")

# Check if file exists
if not os.path.exists(sa_demand_and_prices_path):
    print("sa_demand_and_prices.csv not found. Querying database ...")

    # Get generation and price data
    sa_demand_and_prices = get_mms_data(
        script_name="collect_sa_demand_and_prices",
        arguments={
            "start_date": "2020-01-01",
            "end_date": "2024-12-31"})
    sa_demand_and_prices.set_index("Interval", inplace=True)

    # Save data to file
    sa_demand_and_prices.to_csv(sa_demand_and_prices_path)
    print(f"Data saved successfully.")

else:
    print(f"Loading existing sa_demand_and_prices.csv file ...")
    # Load existing data
    sa_demand_and_prices = pd.read_csv(
        sa_demand_and_prices_path,
        parse_dates=[0],
        index_col=0)

# Display market data
print("Data loaded successfully.")
sa_demand_and_prices

## Observations of SA wind and solar market

Calculate and visualise metrics of:
* Size of the wind and solar fleets over time (2020 to 2024)
* Capacity factors of wind and solar generators
* Intraday shape of wind and solar generation, and load
* Demand-weighted prices for SA (whole of market), wind and solar

### Size of wind and solar fleets

In [ ]:
# Identify changes in DUID capacity:
for duid in gen_info.index:
    name = gen_info.at[duid, "Name"]
    duid_data = sa_duid_data[sa_duid_data["DUID"] == duid]
    capacity_readings = len(duid_data["Maximum capacity"].unique())
    if capacity_readings > 1:
        print(f"{duid} ({name}) has {capacity_readings} values of maximum capacity:")


### Demand-weighted prices

In [ ]:
sa_demand_and_prices["Total value"] = sa_demand_and_prices.iloc[:, :2].product(axis=1)
sa_demand_and_prices_qtr = sa_demand_and_prices.resample("QE").sum()
sa_demand_and_prices_qtr["DWA price"] = sa_demand_and_prices_qtr["Total value"] / sa_demand_and_prices_qtr["Total demand"]
sa_demand_and_prices_qtr